# Putting the G(enerate) in RAG

In [37]:
from openai import OpenAI
from datetime import datetime
import hashlib
import re
import os
from sentence_transformers import CrossEncoder


from tqdm import tqdm
import numpy as np
from torch import nn

import logging
from pinecone import Pinecone, ServerlessSpec

logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)


In [44]:
from pydantic import BaseModel, Field
from typing import List, Dict, Tuple

In [45]:
# Initialize the OpenAI client with the API key from user data
client = OpenAI(api_key=os.environ.get('OPENAI_API_KEY'))


In [46]:
# Define a class for the Chat Language Model
class ChatLLM(BaseModel):
    model: str = 'gpt-4o'  # Default model to use
    temperature: float = 0.0  # Default temperature for generating responses

    # Method to generate a response from the model based on the provided prompt
    def generate(self, prompt: str, stop: List[str] = None):
        # Create a completion request to the OpenAI API with the given parameters
        response = client.chat.completions.create(
            model=self.model,
            messages=[{"role": "user", "content": prompt}],
            temperature=self.temperature,
            stop=stop
        )

        # Return the generated response content
        return response.choices[0].message.content


In [47]:
c = ChatLLM()
c.generate('hi')

'Hello! How can I assist you today?'

In [48]:
pinecone_key = os.environ.get('PINECONE_API_KEY')
INDEX_NAME = 'semantic-search-test'
ENGINE = 'text-embedding-3-large'
NAMESPACE = 'default'

pc = Pinecone(
    api_key=pinecone_key
)

# helper functions to get lists of embeddings from the OpenAI API
def get_embedding(text, engine=ENGINE):
    response = client.embeddings.create(
        input=[text],
        model=engine
    )
    return response.data[0].embedding

len(get_embedding('hi'))

3072

In [49]:
# Store the index as a variable
index = pc.Index(name=INDEX_NAME)
index

In [50]:
def query_from_pinecone(query, top_k=1, include_metadata=True):
    # get embedding from THE SAME embedder as the documents
    query_embedding = get_embedding(query, engine=ENGINE)

    return index.query(
      vector=query_embedding,
      top_k=top_k,
      namespace=NAMESPACE,
      include_metadata=include_metadata   # gets the metadata (dates, text, etc)
    ).get('matches')

query_from_pinecone('What is the X-24?')[0]['metadata']

{'date_uploaded': '2024-02-04T15:47:12.501860',
 'text': 'The X-24A was a fat, short teardrop shape with vertical fins for control. It made its first, unpowered, glide flight on April 17, 1969 with Air Force Maj. Jerauld R. Gentry at the controls. Gentry also piloted its first powered flight on March 19, 1970. The craft was taken to around 45,000 feet (13.7 km) by a modified B-52 and then drop launched, then either glided down or used its rocket engine to ascend to higher altitudes before gliding down. The X-24A was flown 28 times at speeds up to 1,036 mph (1,667 km/h) and altitudes up to 71,400 feet (21.8 km).'}

In [60]:
FINAL_ANSWER_TOKEN = "Assistant Response:"
STOP = '[END]'
PROMPT_TEMPLATE = """Today is {today} and you can retrieve information from a database. Respond the user's input as best as you can.

Here is an example of the conversation format:

[START]
User Input: the input question you must answer
Context: retrieved context from the database
Context Score : a score from 0 - 1 of how strong the information is a match
Assistant Thought: This context has sufficient information to answer the question.
Assistant Response: your final answer to the original input question which could be I don't have sufficient information to answer the question.
[END]
[START]
User Input: another input question you must answer
Context: more retrieved context from the database
Context Score : another score from 0 - 1 of how strong the information is a match
Assistant Thought: This context does not have sufficient information to answer the question.
Assistant Response: your final answer to the second input question which could be I don't have sufficient information to answer the question.
[END]
[START]
User Input: another input question you must answer
Context: NO CONTEXT FOUND
Context Score : 0
Assistant Thought: We either could not find something or we don't need to look something up
Assistant Response: Reason through the best way to answer
[END]

Begin:

{running_convo}
"""

class RagBot(BaseModel):
    llm: ChatLLM
    prompt_template: str = PROMPT_TEMPLATE
    stop_pattern: List[str] = [STOP]
    user_inputs: List[str] = []
    ai_responses: List[str] = []
    contexts: List[Tuple[str, float]] = []
    verbose: bool = False
    threshold: float = 0.6

    def query_from_pinecone(self, query, top_k=1, include_metadata=True):
        return query_from_pinecone(query, top_k, include_metadata)

    @property
    def running_convo(self):
        convo = ''
        for index in range(len(self.user_inputs)):
            convo += f'[START]\nUser Input: {self.user_inputs[index]}\n'
            convo += f'Context: {self.contexts[index][0]}\nContext Score: {self.contexts[index][1]}\n'
            if len(self.ai_responses) > index:
                convo += self.ai_responses[index]
                convo += '\n[END]\n'
        return convo.strip()

    def run(self, question: str):
        self.user_inputs.append(question)
        top_response = self.query_from_pinecone(question)[0]
        if top_response['score'] >= self.threshold:
            self.contexts.append(
                (top_response['metadata']['text'], top_response['score']))
        else:
            self.contexts.append(('NO CONTEXT FOUND', 'NONE', 0))

        prompt = self.prompt_template.format(
                today = datetime.today(),
                running_convo=self.running_convo
        )
        if self.verbose:
            print('--------')
            print('PROMPT')
            print('--------')
            print(prompt)
            print('--------')
            print('END PROMPT')
            print('--------')
        generated = self.llm.generate(prompt, stop=self.stop_pattern)
        if self.verbose:
            print('--------')
            print('GENERATED')
            print('--------')
            print(generated)
            print('--------')
            print('END GENERATED')
            print('--------')
        self.ai_responses.append(generated)
        if FINAL_ANSWER_TOKEN in generated:
            generated = generated.split(FINAL_ANSWER_TOKEN)[-1]
        return generated

In [84]:
r = RagBot(llm=ChatLLM(temperature=0.0), stop_pattern=['[END]'])
print(r.run('What is the X-24?'))

 The X-24A was an experimental aircraft with a fat, short teardrop shape and vertical fins for control. It made its first unpowered glide flight on April 17, 1969, piloted by Air Force Maj. Jerauld R. Gentry, who also piloted its first powered flight on March 19, 1970. The X-24A was drop launched from a modified B-52 at around 45,000 feet and could either glide down or use its rocket engine to ascend to higher altitudes before gliding down. It was flown 28 times, reaching speeds up to 1,036 mph and altitudes up to 71,400 feet.


In [85]:
print(r.run('Who is Obama?'))

 Barack Obama is a former President of the United States, having served two terms from January 20, 2009, to January 20, 2017. He was the first African American to hold the office. Before his presidency, he was a U.S. Senator from Illinois. He is a member of the Democratic Party.


In [86]:
print(r.running_convo)

[START]
User Input: What is the X-24?
Context: The X-24A was a fat, short teardrop shape with vertical fins for control. It made its first, unpowered, glide flight on April 17, 1969 with Air Force Maj. Jerauld R. Gentry at the controls. Gentry also piloted its first powered flight on March 19, 1970. The craft was taken to around 45,000 feet (13.7 km) by a modified B-52 and then drop launched, then either glided down or used its rocket engine to ascend to higher altitudes before gliding down. The X-24A was flown 28 times at speeds up to 1,036 mph (1,667 km/h) and altitudes up to 71,400 feet (21.8 km).
Context Score: 0.686325669
Assistant Thought: This context has sufficient information to answer the question.
Assistant Response: The X-24A was an experimental aircraft with a fat, short teardrop shape and vertical fins for control. It made its first unpowered glide flight on April 17, 1969, piloted by Air Force Maj. Jerauld R. Gentry, who also piloted its first powered flight on March 19,

# Attempting to use Llama-3 as our Generator

In [64]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [73]:
import requests

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>"),
    tokenizer.convert_tokens_to_ids("assistant"),

]

def test_prompt_llama_3_8b(prompt, suppress=False, **kwargs):

    API_URL = "https://my03m9749ssz7t6h.us-east-1.aws.endpoints.huggingface.cloud"
    headers = {
    	"Accept" : "application/json",
    	"Authorization": f"Bearer {os.environ.get('HF_TOKEN')}",
    	"Content-Type": "application/json"
    }

    llama_prompt = f"<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\n{prompt}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"

    def query(payload):
    	response = requests.post(API_URL, headers=headers, json=payload)
    	return response.json()

    kwargs["return_text"] = False
    kwargs["return_full_text"] = False
    kwargs['max_new_tokens'] = 512
    kwargs['stop'] = ["<|end_of_text|>", "<|eot_id|>"]

    output = query({
    	"inputs": llama_prompt,
    	"parameters": kwargs
    })
    answer = output[0]['generated_text']
    if not suppress:
        print(f'PROMPT:\n------\n{llama_prompt}\n------\nRESPONSE\n------\n{answer}')
    else:
        return answer

test_prompt_llama_3_8b('1+1=?')

PROMPT:
------
<|begin_of_text|><|start_header_id|>user<|end_header_id|>

1+1=?<|eot_id|><|start_header_id|>assistant<|end_header_id|>


------
RESPONSE
------
2


In [74]:
test_prompt_llama_3_8b('1+1=?', suppress=True)

'2'

In [75]:
class LlamaChatLLM(ChatLLM):
    temperature: float = 0.3
    do_sample: bool = True
    max_new_tokens: int = 256

    def generate(self, prompt: str, stop: List[str] = None):
        response = test_prompt_llama_3_8b(prompt, suppress=True)
        return response

In [87]:
llama_rag = RagBot(llm=llama_llm, verbose=False, stop_pattern=['[END]'])
print(llama_rag.run('What is the X-24?'))

 The X-24A was a experimental aircraft that made its first flight in 1969 and was flown 28 times, reaching speeds of up to 1,036 mph and altitudes of up to 71,400 feet.


# Attempting to use [Command-R](https://cohere.com/blog/command-r?ref=cohere-ai.ghost.io) as our Generator

In [ ]:
# https://huggingface.co/CohereForAI/c4ai-command-r-v01-4bit

In [ ]:
!pip install bitsandbytes accelerate torch[transformers]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 30.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-man

In [ ]:
from transformers import AutoTokenizer

model_id = "CohereForAI/c4ai-command-r-v01"
tokenizer = AutoTokenizer.from_pretrained(model_id)

# define conversation input:
conversation = [
    {"role": "user", "content": "Whats the biggest penguin in the world?"}
]
# define documents to ground on:
documents = [
    { "title": "Tall penguins", "text": "Emperor penguins are the tallest growing up to 122 cm in height." },
    { "title": "Penguin habitats", "text": "Emperor penguins only live in Antarctica."}
]

# render the tool use prompt as a string:
grounded_generation_prompt = tokenizer.apply_grounded_generation_template(
    conversation,
    documents=documents,
    citation_mode="accurate", # or "fast"
    tokenize=False,
    add_generation_prompt=True,
)
print(grounded_generation_prompt)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


<BOS_TOKEN><|START_OF_TURN_TOKEN|><|SYSTEM_TOKEN|># Safety Preamble
The instructions in this section override those in the task description and style guide sections. Don't answer questions that are harmful or immoral.

# System Preamble
## Basic Rules
You are a powerful conversational AI trained by Cohere to help people. You are augmented by a number of tools, and your job is to use and consume the output of these tools to best help the user. You will see a conversation history between yourself and a user, ending with an utterance from the user. You will then see a specific instruction instructing you what kind of response to generate. When you answer the user's requests, you cite your sources in your answers, according to those instructions.

# User Preamble
## Task and Context
You help people answer their questions and other requests interactively. You will be asked a very wide array of requests on all kinds of topics. You will be equipped with a wide range of search engines or simil

In [ ]:
# render the tool use prompt as a string:
grounded_generation_tokens = tokenizer.apply_grounded_generation_template(
    conversation,
    documents=documents,
    citation_mode="accurate", # or "fast"
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt"
)
print(grounded_generation_tokens.shape)


torch.Size([1, 579])


In [ ]:
# pip install 'transformers>=4.39.1' bitsandbytes accelerate
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = "CohereForAI/c4ai-command-r-v01-4bit"
model = AutoModelForCausalLM.from_pretrained(model_id)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
gen_tokens = model.generate(
    grounded_generation_tokens,
    max_new_tokens=512,
    do_sample=True,
    temperature=0.3,
    )

gen_text = tokenizer.decode(gen_tokens[0])
print(gen_text)


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1659: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


<BOS_TOKEN><|START_OF_TURN_TOKEN|><|SYSTEM_TOKEN|># Safety Preamble
The instructions in this section override those in the task description and style guide sections. Don't answer questions that are harmful or immoral.

# System Preamble
## Basic Rules
You are a powerful conversational AI trained by Cohere to help people. You are augmented by a number of tools, and your job is to use and consume the output of these tools to best help the user. You will see a conversation history between yourself and a user, ending with an utterance from the user. You will then see a specific instruction instructing you what kind of response to generate. When you answer the user's requests, you cite your sources in your answers, according to those instructions.

# User Preamble
## Task and Context
You help people answer their questions and other requests interactively. You will be asked a very wide array of requests on all kinds of topics. You will be equipped with a wide range of search engines or simil

In [ ]:
def format_for_command_r(documents):
    return [{'title': f'Document {index + 1}', 'text': document['metadata']['text']} for index, document in enumerate(documents)]

command_r_docs = format_for_command_r(query_from_pinecone('I lost my card', top_k=3, include_metadata=True))

len(command_r_docs)

3

In [ ]:
# define conversation input:
conversation = [
    {"role": "user", "content": "I lost my card"}
]

# render the tool use prompt as a string:
grounded_generation_tokens = tokenizer.apply_grounded_generation_template(
    conversation,
    documents=command_r_docs,
    citation_mode="citation",
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt"
)

gen_tokens = model.generate(
    grounded_generation_tokens,
    max_new_tokens=512,
    do_sample=True,
    temperature=0.3,
    )

gen_text = tokenizer.decode(gen_tokens[0])
print(gen_text)


<BOS_TOKEN><|START_OF_TURN_TOKEN|><|SYSTEM_TOKEN|># Safety Preamble
The instructions in this section override those in the task description and style guide sections. Don't answer questions that are harmful or immoral.

# System Preamble
## Basic Rules
You are a powerful conversational AI trained by Cohere to help people. You are augmented by a number of tools, and your job is to use and consume the output of these tools to best help the user. You will see a conversation history between yourself and a user, ending with an utterance from the user. You will then see a specific instruction instructing you what kind of response to generate. When you answer the user's requests, you cite your sources in your answers, according to those instructions.

# User Preamble
## Task and Context
You help people answer their questions and other requests interactively. You will be asked a very wide array of requests on all kinds of topics. You will be equipped with a wide range of search engines or simil